# QR Factorisation

We will look at two different ways for computing the QR factorisation of a matrix. To goal is to start from a matrix $A$ and write it as the product of an orthogonal matrix $Q$ and an upper-triangular matrix $R$.

In [3]:
import numpy as np
from numpy.linalg import norm

A = np.array([[1,2,3],[4,5,6],[7,8,10]])
b = np.array([1,2,3])
n = 3

In [4]:
A

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8, 10]])

Numpy has a built-in function for doing this. Let's use it to check what the answer should be:

In [29]:
(Q,R) = np.linalg.qr(A)

array([[ -8.1240384 ,  -9.6011363 , -11.93987462],
       [  0.        ,   0.90453403,   1.50755672],
       [  0.        ,   0.        ,   0.40824829]])

In [30]:
Q

array([[-0.12309149,  0.90453403,  0.40824829],
       [-0.49236596,  0.30151134, -0.81649658],
       [-0.86164044, -0.30151134,  0.40824829]])

In [31]:
R

array([[ -8.1240384 ,  -9.6011363 , -11.93987462],
       [  0.        ,   0.90453403,   1.50755672],
       [  0.        ,   0.        ,   0.40824829]])

## Gram-Schmidt orthogonalization

The first approach will be to transform the vectors in the columns of $A$ to a set of orthogonal vectores using the Gram-Schmidt approach. The basic idea of Gram-Schmidt is to build up an orthonormal set of vectors by projecting out non-orthogonal pieces. The following image illustrates this.
![Gram-Schmidt Visualisation](Gram-Schmidt_orthonormalization_process.gif "Gram-Schmidt Visualisation")
Let's now implement this with our test matrix $A$.

1. First, we construct three vectors $a_1$, $a_2$ and $a_3$ from the columns of $A$.

In [42]:
#a1 = A[:,0]
#a2 = A[:,1]
#a3 = A[:,2]

# alternatively
(a1,a2,a3) = np.transpose(A)

2. Now, our first orthonormal vector is just $a_1$ normalised to have length 1:

In [85]:
e1 = a1/norm(a1)

3. To construct our second orthonormal vector, let's start with $a_2$, project out the part along the $a_1$ direction and normalise the result:

In [86]:
a2proj = (a2 - (e1@a2)*e1)
e2 = a2proj/norm(a2proj)

4. Let's also project this piece out from $a_3$ now (this is not essential, but helps improve the numerical stability of the algorithm)

In [87]:
a3proj = (a3 - (e1@a3)*e1)

5. To construct our third orthonormal vector, let's project out the part along the previous two directions and normalise the result:

In [88]:
a3proj2 = a3proj - (e2@a3proj)*e2
e3 = a3proj2/norm(a3proj2)
e3

array([ 0.40824829, -0.81649658,  0.40824829])

6. Now we have our three orthogonal vectors, we can put them into the columns of Q

In [90]:
Q = np.zeros((3,3))
Q = np.transpose((e1,e2,e3))
Q

array([[ 0.12309149,  0.90453403,  0.40824829],
       [ 0.49236596,  0.30151134, -0.81649658],
       [ 0.86164044, -0.30151134,  0.40824829]])

7. To get $R$, we note that $A = Q R$ means that $Q^T A = Q^T Q R = R$ since $Q$ is an orthogonal matrix. Let's use this to compute $R$:

In [91]:
R=np.transpose(Q)@A
R

array([[8.12403840e+00, 9.60113630e+00, 1.19398746e+01],
       [1.19348975e-14, 9.04534034e-01, 1.50755672e+00],
       [1.55986335e-14, 1.86517468e-14, 4.08248290e-01]])

8. As expected, $R$ is (almost) an upper-triangular matrix. It is only __almost__ upper triangular because floating point arithmetic is not exact.

## Using Householder reflections

The Gram-Schmidt process can be a very effective way to orthogonalise a set of vectors, but it does run into problems with numerical stability. This can happen, for example, in the case where we are starting with vectors that are nearly linearly dependent. Then we would be subtracting two large vectors to produce one small one, and we know that this is a recipe for disaster with floating point arithmetic.

One way around this problem is to use a different approach to orthogonalization. A very popular method uses the idea of Householder reflections. These take a vector $x$ and reflect it about a plane defined by another vector $v$:
![Householder reflection](Householder.png "Householder reflection")
This is clearly equivalent to multiplying $x$ by the __Householder reflection matrix__
$$ H = I - 2 \frac{v v^T}{||v||^2}$$
Note that $H$ is a *symmetric, orthogonal matrix*.

Now, if we choose $v$ appropriately then we can use it to zero out below the pivot in each column, thus producing $R$. In particular, if
$$v = a - ||a|| e_k$$ then $H a = |a| e_k$ so if $e_k$ is a unit vector in the $k$-th direction this does exactly what we want to the column $a$.

Let's now implement this in practice.

First, we work on the first column of $A$.

In [129]:
u1 = a1 - (-np.sign(a1[0]))*norm(a1)*np.array([1,0,0])
v1 = u1/norm(u1)
H1 = np.identity(n) - 2*np.outer(v1,v1)

In [130]:
H1

array([[-0.12309149, -0.49236596, -0.86164044],
       [-0.49236596,  0.7841456 , -0.3777452 ],
       [-0.86164044, -0.3777452 ,  0.33894589]])

In [131]:
A1 = H1@A
A1

array([[-8.12403840e+00, -9.60113630e+00, -1.19398746e+01],
       [ 7.21644966e-16, -8.59655700e-02, -5.49676344e-01],
       [ 7.77156117e-16, -9.00439748e-01, -1.46193360e+00]])

and the second column

In [132]:
a2 = A1[1:,1]
u2 = a2 - (-np.sign(a2[0]))*norm(a2)*np.array([1,0])
v2 = u2/norm(u2)
H2 = np.identity(n)
H2[1:,1:] -= 2*np.outer(v2,v2)
A2=H2@A1
A2

array([[-8.12403840e+00, -9.60113630e+00, -1.19398746e+01],
       [-8.42222460e-16,  9.04534034e-01,  1.50755672e+00],
       [-6.44518747e-16,  2.17887053e-17,  4.08248290e-01]])

and finally the third column

In [133]:
a3= A2[2:,2]
u3 = a3 - (-np.sign(a3[0]))*norm(a3)*np.array([1])
v3 = u3/norm(u3)
H3 = np.identity(n)
H3[2:,2:] -= 2*np.outer(v3,v3)
A3=H3@A2
A3

array([[-8.12403840e+00, -9.60113630e+00, -1.19398746e+01],
       [-8.42222460e-16,  9.04534034e-01,  1.50755672e+00],
       [ 6.44518747e-16, -2.17887053e-17, -4.08248290e-01]])

We now have transformed to exactly $R$.

In [134]:
np.transpose(H3@H2@H1)

array([[-0.12309149,  0.90453403, -0.40824829],
       [-0.49236596,  0.30151134,  0.81649658],
       [-0.86164044, -0.30151134, -0.40824829]])

And we can easily construct $Q$ from the Householder matrices

In [115]:
Q

array([[ 0.12309149,  0.90453403,  0.40824829],
       [ 0.49236596,  0.30151134, -0.81649658],
       [ 0.86164044, -0.30151134,  0.40824829]])